# 📊 Book de Variáveis - Recarga CMV

## Contexto
Book com **100+ variáveis** para modelo de **Inadimplência** em clientes que migram de PRE para CONTROLE.

## Granularidade
- **Chave**: `SAFRA` (yyyyMM de `DAT_INSERCAO_CREDITO`) + `NUM_CPF`
- **Janelas**: Safra atual, 3M, 6M

## Tabelas
| Tabela | Tipo |
|--------|------|
| `rawdata.ass_recarga_cmv_nova` | Fato |
| `rawdata.canal_aquisicao_credito` | Dimensão |
| `rawdata.plano_preco` | Dimensão |
| `rawdata.tipo_recarga` | Dimensão |
| `rawdata.tipo_insercao` | Dimensão |
| `rawdata.forma_pagamento` | Dimensão |
| `rawdata.instituicao` | Dimensão |

In [ ]:
# =============================================================================
# PARAMETROS DO NOTEBOOK (SAFRAS)
# =============================================================================
from datetime import date

LISTA_SAFRAS = ['202410', '202411', '202412', '202501', '202502', '202503']
MODO_ESCRITA = 'overwrite'

def safra_to_cutoff(safra):
    '''Converte safra (yyyyMM) para data de cutoff.'''
    return date(int(safra[:4]), int(safra[4:6]), 1).strftime('%Y-%m-%d')

print('Configuracao de Safras:')
for s in LISTA_SAFRAS:
    print(f'  Safra {s}: dados < {safra_to_cutoff(s)}')

In [ ]:
# =============================================================================
# CONFIGURAÇÃO DO AMBIENTE
# =============================================================================

LAKEHOUSE_PATH = "abfss://febb8631-d5c0-43d8-bf08-5e89c8f2d17e@onelake.dfs.fabric.microsoft.com/5f8a4808-6f65-401b-a427-b0dd9d331b35"

# Paths de Entrada
PATH_RECARGA = f"{LAKEHOUSE_PATH}/Tables/rawdata/ass_recarga_cmv_nova"
PATH_CANAL = f"{LAKEHOUSE_PATH}/Tables/rawdata/canal_aquisicao_credito"
PATH_PLANO = f"{LAKEHOUSE_PATH}/Tables/rawdata/plano_preco"
PATH_TIPO_RECARGA = f"{LAKEHOUSE_PATH}/Tables/rawdata/tipo_recarga"
PATH_TIPO_INSERCAO = f"{LAKEHOUSE_PATH}/Tables/rawdata/tipo_insercao"
PATH_FORMA_PAGAMENTO = f"{LAKEHOUSE_PATH}/Tables/rawdata/forma_pagamento"
PATH_INSTITUICAO = f"{LAKEHOUSE_PATH}/Tables/rawdata/instituicao"

# Path de Saída
PATH_OUTPUT = f"{LAKEHOUSE_PATH}/Tables/book/ass_recarga_cmv"

print("✅ Configuração carregada")
print(f"📁 Output: {PATH_OUTPUT}")

In [ ]:
# =============================================================================
# CARREGAMENTO DAS TABELAS DELTA
# =============================================================================

df_recarga = spark.read.format("delta").load(PATH_RECARGA)
print(f"📊 recarga: {df_recarga.count():,} registros")

df_canal = spark.read.format("delta").load(PATH_CANAL)
df_plano = spark.read.format("delta").load(PATH_PLANO)
df_tipo_recarga = spark.read.format("delta").load(PATH_TIPO_RECARGA)
df_tipo_insercao = spark.read.format("delta").load(PATH_TIPO_INSERCAO)
df_forma_pagamento = spark.read.format("delta").load(PATH_FORMA_PAGAMENTO)
df_instituicao = spark.read.format("delta").load(PATH_INSTITUICAO)

print("\n✅ Todas as tabelas carregadas")

In [ ]:
# =============================================================================
# REGISTRO DE VIEWS TEMPORÁRIAS
# =============================================================================

df_recarga.createOrReplaceTempView("recarga")
df_canal.createOrReplaceTempView("dim_canal")
df_plano.createOrReplaceTempView("dim_plano")
df_tipo_recarga.createOrReplaceTempView("dim_tipo_recarga")
df_tipo_insercao.createOrReplaceTempView("dim_tipo_insercao")
df_forma_pagamento.createOrReplaceTempView("dim_forma_pagamento")
df_instituicao.createOrReplaceTempView("dim_instituicao")

print("✅ Views registradas")

In [ ]:
# =============================================================================
# QUERY SQL - BOOK DE VARIÁVEIS DE RECARGA
# =============================================================================

SQL_TEMPLATE = """
WITH 
-- =============================================================================
-- BASE: Enriquecimento com Dimensões
-- =============================================================================
base_enrich AS (
    SELECT 
        r.*,
        '{safra}' AS SAFRA,
        
        -- Dimensão Canal
        c.DSC_CANAL_AQUISICAO_BI,
        c.COD_TIPO_CREDITO AS CANAL_TIPO_CREDITO,
        
        -- Dimensão Plano
        p.DSC_TIPO_PLANO_BI,
        p.DSC_GRUPO_PLANO_BI,
        p.IND_AMDOCS_PLAT_PRE,
        
        -- Dimensão Tipo Recarga
        tr.DSC_TIPO_RECARGA,
        
        -- Dimensão Tipo Inserção
        ti.DSC_TIPO_INSERCAO,
        
        -- Dimensão Forma Pagamento
        fp.DSC_FORMA_PAGAMENTO,
        
        -- Dimensão Instituição
        i.DSC_INSTITUICAO,
        i.DSC_TIPO_INSTITUICAO,
        
        -- Flags Pivotadas: Plataforma (PREPG/AUTOC/FLEXD/CTLFC/POSPG)
        CASE WHEN r.COD_PLATAFORMA_ATU = 'PREPG' THEN 1 ELSE 0 END AS FLAG_PLAT_PREPG,
        CASE WHEN r.COD_PLATAFORMA_ATU = 'AUTOC' THEN 1 ELSE 0 END AS FLAG_PLAT_AUTOC,
        CASE WHEN r.COD_PLATAFORMA_ATU = 'FLEXD' THEN 1 ELSE 0 END AS FLAG_PLAT_FLEXD,
        CASE WHEN r.COD_PLATAFORMA_ATU = 'CTLFC' THEN 1 ELSE 0 END AS FLAG_PLAT_CTLFC,
        CASE WHEN r.COD_PLATAFORMA_ATU = 'POSPG' THEN 1 ELSE 0 END AS FLAG_PLAT_POSPG,
        
        -- Flags Pivotadas: Status Plataforma (A/ZB1/ZB2/NDF)
        CASE WHEN r.COD_STATUS_PLATAFORMA = 'A' THEN 1 ELSE 0 END AS FLAG_STATUS_A,
        CASE WHEN r.COD_STATUS_PLATAFORMA = 'ZB1' THEN 1 ELSE 0 END AS FLAG_STATUS_ZB1,
        CASE WHEN r.COD_STATUS_PLATAFORMA = 'ZB2' THEN 1 ELSE 0 END AS FLAG_STATUS_ZB2,
        CASE WHEN r.COD_STATUS_PLATAFORMA = 'NDF' THEN 1 ELSE 0 END AS FLAG_STATUS_NDF,
        
        -- Flags Pivotadas: Grupo Cartão WPP
        CASE WHEN r.DSC_GRUPO_CARTAO_WPP = 'NaoSeAplica' THEN 1 ELSE 0 END AS FLAG_CARTAO_NA,
        CASE WHEN r.DSC_GRUPO_CARTAO_WPP = 'Rec.Online' THEN 1 ELSE 0 END AS FLAG_CARTAO_ONLINE,
        CASE WHEN r.DSC_GRUPO_CARTAO_WPP = 'AtivPromocao' THEN 1 ELSE 0 END AS FLAG_CARTAO_PROMO,
        CASE WHEN r.DSC_GRUPO_CARTAO_WPP LIKE 'ChipPre%' THEN 1 ELSE 0 END AS FLAG_CARTAO_CHIPPRE,
        
        -- Flags Pivotadas: Tipo Plano BI
        CASE WHEN p.DSC_TIPO_PLANO_BI = 'Varejo' THEN 1 ELSE 0 END AS FLAG_PLANO_VAREJO,
        CASE WHEN p.DSC_TIPO_PLANO_BI = 'Corporativos' THEN 1 ELSE 0 END AS FLAG_PLANO_CORP,
        CASE WHEN p.DSC_TIPO_PLANO_BI = 'Mid' THEN 1 ELSE 0 END AS FLAG_PLANO_MID,
        
        -- Flags Pivotadas: Tipo Instituição
        CASE WHEN i.DSC_TIPO_INSTITUICAO = 'Distribuidor Regional' THEN 1 ELSE 0 END AS FLAG_INST_DIST_REG,
        CASE WHEN i.DSC_TIPO_INSTITUICAO = 'Street Seller' THEN 1 ELSE 0 END AS FLAG_INST_STREET,
        CASE WHEN i.DSC_TIPO_INSTITUICAO = 'Venda Direta' THEN 1 ELSE 0 END AS FLAG_INST_DIRETA,
        CASE WHEN i.DSC_TIPO_INSTITUICAO = 'Varejo' THEN 1 ELSE 0 END AS FLAG_INST_VAREJO,
        
        -- Flag SOS
        COALESCE(r.FLAG_SOS, 0) AS FLAG_SOS_CLEAN,
        
        -- Indicador de Migração PRE->CONTROLE
        CASE WHEN r.COD_PLATAFORMA_ATU IN ('CTLFC', 'FLEXD') THEN 1 ELSE 0 END AS FLAG_PLAT_CONTROLE
        
    FROM recarga r
    
    LEFT JOIN dim_canal c 
        ON r.COD_CANAL_AQUISICAO = c.COD_CANAL_AQUISICAO
        
    LEFT JOIN dim_plano p 
        ON r.DW_PLANO_TARIFACAO = p.DW_PLANO
        
    LEFT JOIN dim_tipo_recarga tr 
        ON r.DW_TIPO_RECARGA = tr.DW_TIPO_RECARGA
        
    LEFT JOIN dim_tipo_insercao ti 
        ON r.DW_TIPO_INSERCAO = ti.DW_TIPO_INSERCAO
        
    LEFT JOIN dim_forma_pagamento fp 
        ON r.DW_FORMA_PAGAMENTO = fp.DW_FORMA_PAGAMENTO
        
    LEFT JOIN dim_instituicao i 
        ON r.DW_INSTITUICAO = i.DW_INSTITUICAO
    WHERE r.DAT_INSERCAO_CREDITO < '{data_cutoff}'
),

-- =============================================================================
-- AGREGAÇÃO PRINCIPAL: Por SAFRA + NUM_CPF
-- =============================================================================
agregado AS (
    SELECT 
        SAFRA,
        NUM_CPF,
        
        -- =====================================================================
        -- SEÇÃO 1: VOLUMETRIA
        -- =====================================================================
        COUNT(*) AS QTD_RECARGAS_TOTAL,
        COUNT(DISTINCT DW_NUM_NTC) AS QTD_LINHAS,
        COUNT(DISTINCT DW_NUM_CLIENTE) AS QTD_CLIENTES_DW,
        COUNT(DISTINCT COD_PLATAFORMA_ATU) AS QTD_PLATAFORMAS,
        COUNT(DISTINCT DW_TIPO_RECARGA) AS QTD_TIPOS_RECARGA,
        COUNT(DISTINCT DW_TIPO_INSERCAO) AS QTD_TIPOS_INSERCAO,
        COUNT(DISTINCT DW_FORMA_PAGAMENTO) AS QTD_FORMAS_PAGTO,
        COUNT(DISTINCT DW_INSTITUICAO) AS QTD_INSTITUICOES,
        COUNT(DISTINCT DW_PLANO_TARIFACAO) AS QTD_PLANOS,
        COUNT(DISTINCT DATE(DAT_INSERCAO_CREDITO)) AS QTD_DIAS_RECARGA,
        
        -- =====================================================================
        -- SEÇÃO 2: VALORES DE CRÉDITO
        -- =====================================================================
        SUM(COALESCE(VAL_CREDITO_INSERIDO, 0)) AS VLR_CREDITO_TOTAL,
        AVG(COALESCE(VAL_CREDITO_INSERIDO, 0)) AS VLR_CREDITO_MEDIO,
        MAX(COALESCE(VAL_CREDITO_INSERIDO, 0)) AS VLR_CREDITO_MAX,
        MIN(CASE WHEN VAL_CREDITO_INSERIDO > 0 THEN VAL_CREDITO_INSERIDO END) AS VLR_CREDITO_MIN,
        STDDEV(COALESCE(VAL_CREDITO_INSERIDO, 0)) AS VLR_CREDITO_STDDEV,
        
        SUM(COALESCE(VAL_BONUS, 0)) AS VLR_BONUS_TOTAL,
        AVG(COALESCE(VAL_BONUS, 0)) AS VLR_BONUS_MEDIO,
        MAX(COALESCE(VAL_BONUS, 0)) AS VLR_BONUS_MAX,
        
        SUM(COALESCE(VAL_REAL, 0)) AS VLR_REAL_TOTAL,
        AVG(COALESCE(VAL_REAL, 0)) AS VLR_REAL_MEDIO,
        MAX(COALESCE(VAL_REAL, 0)) AS VLR_REAL_MAX,
        STDDEV(COALESCE(VAL_REAL, 0)) AS VLR_REAL_STDDEV,
        
        -- =====================================================================
        -- SEÇÃO 3: PIVOT POR PLATAFORMA (PREPG/AUTOC/FLEXD/CTLFC/POSPG)
        -- =====================================================================
        SUM(FLAG_PLAT_PREPG) AS QTD_PLAT_PREPG,
        SUM(FLAG_PLAT_AUTOC) AS QTD_PLAT_AUTOC,
        SUM(FLAG_PLAT_FLEXD) AS QTD_PLAT_FLEXD,
        SUM(FLAG_PLAT_CTLFC) AS QTD_PLAT_CTLFC,
        SUM(FLAG_PLAT_POSPG) AS QTD_PLAT_POSPG,
        
        SUM(CASE WHEN FLAG_PLAT_PREPG = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_PLAT_PREPG,
        SUM(CASE WHEN FLAG_PLAT_AUTOC = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_PLAT_AUTOC,
        SUM(CASE WHEN FLAG_PLAT_FLEXD = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_PLAT_FLEXD,
        SUM(CASE WHEN FLAG_PLAT_CTLFC = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_PLAT_CTLFC,
        
        SUM(FLAG_PLAT_CONTROLE) AS QTD_PLAT_CONTROLE,
        
        -- =====================================================================
        -- SEÇÃO 4: PIVOT POR STATUS PLATAFORMA (A/ZB1/ZB2)
        -- =====================================================================
        SUM(FLAG_STATUS_A) AS QTD_STATUS_A,
        SUM(FLAG_STATUS_ZB1) AS QTD_STATUS_ZB1,
        SUM(FLAG_STATUS_ZB2) AS QTD_STATUS_ZB2,
        SUM(FLAG_STATUS_NDF) AS QTD_STATUS_NDF,
        
        SUM(CASE WHEN FLAG_STATUS_A = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_STATUS_A,
        SUM(CASE WHEN FLAG_STATUS_ZB1 = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_STATUS_ZB1,
        SUM(CASE WHEN FLAG_STATUS_ZB2 = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_STATUS_ZB2,
        
        -- =====================================================================
        -- SEÇÃO 5: PIVOT POR GRUPO CARTÃO WPP
        -- =====================================================================
        SUM(FLAG_CARTAO_NA) AS QTD_CARTAO_NA,
        SUM(FLAG_CARTAO_ONLINE) AS QTD_CARTAO_ONLINE,
        SUM(FLAG_CARTAO_PROMO) AS QTD_CARTAO_PROMO,
        SUM(FLAG_CARTAO_CHIPPRE) AS QTD_CARTAO_CHIPPRE,
        
        SUM(CASE WHEN FLAG_CARTAO_ONLINE = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_CARTAO_ONLINE,
        SUM(CASE WHEN FLAG_CARTAO_PROMO = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_CARTAO_PROMO,
        
        -- =====================================================================
        -- SEÇÃO 6: PIVOT POR TIPO PLANO BI
        -- =====================================================================
        SUM(FLAG_PLANO_VAREJO) AS QTD_PLANO_VAREJO,
        SUM(FLAG_PLANO_CORP) AS QTD_PLANO_CORP,
        SUM(FLAG_PLANO_MID) AS QTD_PLANO_MID,
        
        SUM(CASE WHEN FLAG_PLANO_VAREJO = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_PLANO_VAREJO,
        SUM(CASE WHEN FLAG_PLANO_CORP = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_PLANO_CORP,
        
        -- =====================================================================
        -- SEÇÃO 7: PIVOT POR TIPO INSTITUIÇÃO
        -- =====================================================================
        SUM(FLAG_INST_DIST_REG) AS QTD_INST_DIST_REG,
        SUM(FLAG_INST_STREET) AS QTD_INST_STREET,
        SUM(FLAG_INST_DIRETA) AS QTD_INST_DIRETA,
        SUM(FLAG_INST_VAREJO) AS QTD_INST_VAREJO,
        
        SUM(CASE WHEN FLAG_INST_STREET = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_INST_STREET,
        SUM(CASE WHEN FLAG_INST_DIRETA = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_INST_DIRETA,
        
        -- =====================================================================
        -- SEÇÃO 8: SOS (Recarga Emergencial)
        -- =====================================================================
        SUM(FLAG_SOS_CLEAN) AS QTD_SOS,
        SUM(CASE WHEN FLAG_SOS_CLEAN = 1 THEN COALESCE(VAL_CREDITO_INSERIDO, 0) ELSE 0 END) AS VLR_SOS_TOTAL,
        
        -- =====================================================================
        -- SEÇÃO 9: MÉTRICAS TEMPORAIS
        -- =====================================================================
        MIN(DAT_INSERCAO_CREDITO) AS DT_PRIMEIRA_RECARGA,
        MAX(DAT_INSERCAO_CREDITO) AS DT_ULTIMA_RECARGA,
        DATEDIFF(MAX(DAT_INSERCAO_CREDITO), MIN(DAT_INSERCAO_CREDITO)) AS DIAS_ENTRE_RECARGAS,
        DATEDIFF(TO_DATE('{data_cutoff}'), MAX(DAT_INSERCAO_CREDITO)) AS DIAS_DESDE_ULTIMA_RECARGA,
        DATEDIFF(TO_DATE('{data_cutoff}'), MIN(DAT_INSERCAO_CREDITO)) AS DIAS_DESDE_PRIMEIRA_RECARGA,
        
        COUNT(DISTINCT DATE_FORMAT(DAT_INSERCAO_CREDITO, 'yyyyMM')) AS QTD_MESES_ATIVOS,
        
        -- =====================================================================
        -- SEÇÃO 10: DADOS DAS DIMENSÕES
        -- =====================================================================
        FIRST(DSC_TIPO_PLANO_BI) AS TIPO_PLANO_PRINCIPAL,
        FIRST(DSC_TIPO_INSTITUICAO) AS TIPO_INSTITUICAO_PRINCIPAL,
        FIRST(DSC_GRUPO_CARTAO_WPP) AS GRUPO_CARTAO_PRINCIPAL,
        FIRST(COD_PLATAFORMA_ATU) AS PLATAFORMA_PRINCIPAL
        
    FROM base_enrich
    GROUP BY SAFRA, NUM_CPF
)

-- =============================================================================
-- RESULTADO FINAL COM TAXAS, SCORES E SEGMENTAÇÃO
-- =============================================================================
SELECT 
    a.*,
    
    -- =========================================================================
    -- SEÇÃO 11: TAXAS E PROPORÇÕES
    -- =========================================================================
    
    -- Taxa por Plataforma
    a.QTD_PLAT_PREPG / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_PLAT_PREPG,
    a.QTD_PLAT_AUTOC / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_PLAT_AUTOC,
    a.QTD_PLAT_CONTROLE / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_PLAT_CONTROLE,
    
    -- Taxa por Status
    a.QTD_STATUS_A / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_STATUS_A,
    a.QTD_STATUS_ZB1 / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_STATUS_ZB1,
    a.QTD_STATUS_ZB2 / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_STATUS_ZB2,
    
    -- Taxa por Cartão
    a.QTD_CARTAO_ONLINE / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_CARTAO_ONLINE,
    a.QTD_CARTAO_PROMO / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_CARTAO_PROMO,
    
    -- Taxa SOS
    a.QTD_SOS / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0) AS TAXA_SOS,
    
    -- Share of Wallet
    a.VLR_PLAT_PREPG / NULLIF(a.VLR_CREDITO_TOTAL, 0) AS SHARE_PLAT_PREPG,
    a.VLR_PLAT_AUTOC / NULLIF(a.VLR_CREDITO_TOTAL, 0) AS SHARE_PLAT_AUTOC,
    a.VLR_CARTAO_ONLINE / NULLIF(a.VLR_CREDITO_TOTAL, 0) AS SHARE_CARTAO_ONLINE,
    
    -- =========================================================================
    -- SEÇÃO 12: ÍNDICES DE ESTABILIDADE
    -- =========================================================================
    
    -- Coeficiente de Variação
    a.VLR_CREDITO_STDDEV / NULLIF(a.VLR_CREDITO_MEDIO, 0) AS COEF_VARIACAO_CREDITO,
    a.VLR_REAL_STDDEV / NULLIF(a.VLR_REAL_MEDIO, 0) AS COEF_VARIACAO_REAL,
    
    -- Índice de Concentração
    a.VLR_CREDITO_MAX / NULLIF(a.VLR_CREDITO_TOTAL, 0) AS INDICE_CONCENTRACAO_CREDITO,
    
    -- Ticket Médio por Linha
    a.VLR_CREDITO_TOTAL / NULLIF(a.QTD_LINHAS, 0) AS VLR_TICKET_MEDIO_LINHA,
    
    -- Frequência de Recarga (recargas/dias)
    a.QTD_RECARGAS_TOTAL / NULLIF(GREATEST(a.DIAS_ENTRE_RECARGAS, 1), 0) AS FREQ_RECARGA_DIARIA,
    
    -- Ratio Bônus/Crédito
    a.VLR_BONUS_TOTAL / NULLIF(a.VLR_CREDITO_TOTAL, 0) AS RATIO_BONUS_CREDITO,
    
    -- =========================================================================
    -- SEÇÃO 13: SCORE DE RISCO INADIMPLÊNCIA (0-100)
    -- =========================================================================
    LEAST(100, GREATEST(0,
        -- Componente Migração para Controle (peso 30)
        COALESCE(a.QTD_PLAT_CONTROLE / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0), 0) * 30 +
        -- Componente Status Inativo ZB (peso 25)
        COALESCE((a.QTD_STATUS_ZB1 + a.QTD_STATUS_ZB2) / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0), 0) * 25 +
        -- Componente SOS (emergência financeira) (peso 20)
        LEAST(1, COALESCE(a.QTD_SOS / NULLIF(CAST(a.QTD_RECARGAS_TOTAL AS DOUBLE), 0), 0) * 3) * 20 +
        -- Componente Volatilidade (peso 15)
        LEAST(1, COALESCE(a.VLR_CREDITO_STDDEV / NULLIF(a.VLR_CREDITO_MEDIO, 0), 0)) * 15 +
        -- Componente Concentração (peso 10)
        LEAST(1, COALESCE(a.VLR_CREDITO_MAX / NULLIF(a.VLR_CREDITO_TOTAL, 0), 0)) * 10
    )) AS SCORE_RISCO,
    
    -- FLAGS MANDATÓRIAS
    CASE 
        WHEN a.QTD_PLAT_CONTROLE > a.QTD_RECARGAS_TOTAL * 0.5 THEN 1
        WHEN (a.QTD_STATUS_ZB1 + a.QTD_STATUS_ZB2) > a.QTD_RECARGAS_TOTAL * 0.3 THEN 1
        WHEN a.QTD_SOS > a.QTD_RECARGAS_TOTAL * 0.2 THEN 1
        ELSE 0
    END AS FLAG_ALTO_RISCO,
    
    CASE 
        WHEN a.QTD_PLAT_PREPG = a.QTD_RECARGAS_TOTAL
            AND a.QTD_STATUS_A > a.QTD_RECARGAS_TOTAL * 0.95
            AND a.QTD_SOS = 0
        THEN 1
        ELSE 0
    END AS FLAG_BAIXO_RISCO,
    
    -- SEGMENTO DE RISCO
    CASE 
        WHEN a.QTD_PLAT_CONTROLE > a.QTD_RECARGAS_TOTAL * 0.5
            OR (a.QTD_STATUS_ZB1 + a.QTD_STATUS_ZB2) > a.QTD_RECARGAS_TOTAL * 0.3 THEN 'CRITICO'
        WHEN a.QTD_PLAT_CONTROLE > 0
            OR a.QTD_SOS > a.QTD_RECARGAS_TOTAL * 0.1 THEN 'ALTO'
        WHEN a.QTD_STATUS_ZB1 > 0
            OR a.QTD_SOS > 0 THEN 'MEDIO'
        ELSE 'BAIXO'
    END AS SEGMENTO_RISCO,
    
    -- DATA DE PROCESSAMENTO
    CURRENT_TIMESTAMP() AS DT_PROCESSAMENTO
    
FROM agregado a
"""

print("✅ Query SQL definida")
print("📊 Estimativa: 100+ variáveis")

In [ ]:
# =============================================================================
# LOOP PRINCIPAL - PROCESSAR CADA SAFRA
# =============================================================================
resultados = []

for i, safra in enumerate(LISTA_SAFRAS):
    print(f'\n[{i+1}/{len(LISTA_SAFRAS)}] Processando SAFRA {safra}...')
    data_cutoff = safra_to_cutoff(safra)
    print(f'  Cutoff: {data_cutoff}')
    
    query = SQL_TEMPLATE.format(safra=safra, data_cutoff=data_cutoff)
    
    df_safra = spark.sql(query)
    df_safra.cache()
    cnt = df_safra.count()
    print(f'  Registros: {cnt:,}')
    resultados.append({'safra': safra, 'cnt': cnt})
    
    modo = 'overwrite' if i == 0 and MODO_ESCRITA == 'overwrite' else 'append'
    df_safra.write.format('delta').mode(modo).partitionBy('SAFRA').option('overwriteSchema', 'true' if modo == 'overwrite' else 'false').save(PATH_OUTPUT)
    print(f'  Salvo (modo: {modo})')
    df_safra.unpersist()

print('\n' + '='*50)
print('CONCLUIDO!')
total = sum(r['cnt'] for r in resultados)
print(f'Total: {total:,} registros em {len(LISTA_SAFRAS)} safras')

---
# 📖 Dicionário de Variáveis

| Categoria | Variáveis | Descrição |
|-----------|-----------|----------|
| Volumetria | QTD_RECARGAS_TOTAL, QTD_LINHAS, QTD_DIAS_RECARGA | Contagens gerais |
| Valores | VLR_CREDITO_*, VLR_BONUS_*, VLR_REAL_* | Métricas financeiras |
| Plataforma | QTD_PLAT_*, VLR_PLAT_*, TAXA_PLAT_* | Pivot por plataforma (PREPG/AUTOC/CTLFC) |
| Status | QTD_STATUS_*, VLR_STATUS_*, TAXA_STATUS_* | Pivot por status (A/ZB1/ZB2) |
| Cartão WPP | QTD_CARTAO_*, VLR_CARTAO_*, TAXA_CARTAO_* | Pivot por grupo cartão |
| Plano | QTD_PLANO_*, VLR_PLANO_* | Pivot por tipo plano |
| Instituição | QTD_INST_*, VLR_INST_* | Pivot por tipo instituição |
| SOS | QTD_SOS, VLR_SOS_TOTAL, TAXA_SOS | Recargas emergenciais |
| Temporal | DT_*, DIAS_*, QTD_MESES_* | Métricas de tempo |
| Índices | COEF_VARIACAO, INDICE_CONCENTRACAO, SHARE_* | Estabilidade e proporções |
| Scores | SCORE_RISCO (0-100), FLAG_ALTO/BAIXO_RISCO, SEGMENTO_RISCO | Classificação de risco inadimplência |